In [ ]:
############################################################################################
# IMPORTS
############################################################################################
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay


import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
#import tensorflow.keras as keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers


from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



from IPython import display
from PIL import Image


import pathlib
import shutil
import tempfile
import concurrent

import pickle
import visualkeras

import json
import glob






print(tf.__version__)
print(tf.config.list_physical_devices())
############################################################################################

In [ ]:
############################################################################################
# 1. CONSTANTS - PATHS
############################################################################################

#IMAGES = 'iFood_2019'
DATASET = 'food-101'
DATA_FOLDER = './data/' + DATASET
CLASSES_FILE_NAME = '/formated_annot/classes_formated.csv'

TRAIN_PICS_PATH = DATA_FOLDER + '/images/'
#TEST_PICS_PATH = DATA_FOLDER + '/images/'
#VAL_PICS_PATH = DATA_FOLDER + '/images/'

MODELS = './models/'

SEED = 111

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
gpu_dev = tf.config.experimental.list_physical_devices('GPU')
for itm in gpu_dev:
    tf.config.experimental.set_memory_growth(itm, True)


############################################################################################
# IMPORTING DATA
############################################################################################
df_classes = pd.read_csv(DATA_FS251 + CLASSES_FILE_NAME)
df_train = pd.read_csv(DATA_FS251 + TRAIN_INFO, names=['file_name', 'class_num'])
df_validate = pd.read_csv(DATA_FS251 + VAL_INFO, names=['file_name', 'class_num'])
df_test = pd.read_csv(DATA_FS251 + TEST_INFO, names=['file_name'])

df_train.head(3)

In [ ]:
############################################################################################
# 2. CONSTANTS - MODEL
############################################################################################
training_history = dict()

#N_TRAIN = len(df_train.iloc[:, 0])
EPOCHS = 20
BATCH_SIZE = 32              # 8 fit into GPU RAM, 64 fit into system RAM
RESIZE_TO = (512, 512)
#STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE

############################################################################################
# FUNCTIONS, DEFFINITIONS
############################################################################################

class CRelu(tf.keras.Layer):
    def __init__(self, axis=-1, **kwargs):
        self.axis = axis 
        super(CRelu, self).__init__(**kwargs)

    def build(self, input_shape):
        super(CRelu, self).build(input_shape)

    def call(self, x):
        x = tf.nn.crelu(x, axis=self.axis)
        return x

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] = output_shape[-1] * 2
        output_shape = tuple(output_shape)
        return output_shape

    def get_config(self, input_shape):
        config = {'axis': self.axis, }
        base_config = super(CReLU, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
############################################################################################
# import image files
############################################################################################

train_pics, val_pics = tf.keras.preprocessing.image_dataset_from_directory(
    './data/food-101/images/',
    labels='inferred',
#    label_mode='int',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset='both',
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format='channels_last',
    verbose=True
)
#test_pics = tf.keras.preprocessing.image_dataset_from_directory(
#    VAL_PICS_PATH,
#    labels='inferred',
#    label_mode='int',
#    label_mode='categorical',
#    class_names=None,
#    color_mode='rgb',
#    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
#    shuffle=True,
#    seed=SEED,
#    validation_split=None,
#    subset=None,
#    interpolation='bilinear',
#    follow_links=False,
#    crop_to_aspect_ratio=False,
#    pad_to_aspect_ratio=False,
#    data_format='channels_last',
#    verbose=True
#)
#true_test_pics = tf.keras.preprocessing.image_dataset_from_directory(
#    TEST_PICS_PATH,
#    labels=None,
#    label_mode=None,
#    class_names=None,
#    color_mode='rgb',
#    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
#    shuffle=True,
#    seed=SEED,
#    validation_split=None,
#    subset=None,
#    interpolation='bilinear',
#    follow_links=False,
#    crop_to_aspect_ratio=False,
#    pad_to_aspect_ratio=False,
#    data_format='channels_last',
#    verbose=True
#)

train_pics.class_names

In [ ]:
############################################################################################
# MODEL LOAD
############################################################################################
#MODEL_NAME = 'food-101-EfNeetB0-A0.90-E20-earlystop-E4-B16'
MODEL_NAME = 'food-101-EfNeetB0-A0.91-E20'
with tf.device('/GPU:0'):
    model_supclass_loaded = tf.keras.models.load_model(MODELS + MODEL_NAME + '/'+ MODEL_NAME + ".keras")

model_supclass_loaded.summary()



In [ ]:
STOP

In [ ]:
tf.keras.utils.plot_model(
    model_supclass_loaded,
#    to_file='./images/' + MODEL_NAME + '_model_plot.png',
    show_shapes=True,
    show_dtype=True,
    show_layer_names=False,
    rankdir='TB',
    expand_nested=False,
    dpi=200,
    show_layer_activations=True,
    
    show_trainable=True
)

In [ ]:
import visualkeras
from collections import defaultdict
from tensorflow.python.keras.layers import *

color_map = defaultdict(dict)
color_map[Conv2D]['fill'] = 'orange'
color_map[ZeroPadding2D]['fill'] = 'gray'
color_map[Dropout]['fill'] = 'pink'
color_map[MaxPooling2D]['fill'] = 'red'
color_map[Dense]['fill'] = 'green'
color_map[Flatten]['fill'] = 'teal'
visualkeras.layered_view(model_supclass_loaded,
#                        to_file='./images/' + MODEL_NAME + '_layer_plot.png',
                        legend=True,
                        max_xy=250,
                        min_z = 1, 
                        max_z= 75,
                        scale_z = 5,
                        color_map = color_map,
#                        show_nested = True,
                        spacing=50
#                        show_dimension=True
                        )
#visualkeras.graph_view(model_supclass_loaded)

In [ ]:
len(model_supclass_loaded.layers[2].layers)

In [ ]:
############################################################################################
# EVALUATION - ACCURACY
############################################################################################
model_history_df = pd.read_csv(MODELS + MODEL_NAME + '/'+ MODEL_NAME + '.hist_csv')

plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history_df['accuracy'],label='Train accuracy')
plt.plot(model_history_df['val_accuracy'],label='Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
#plt.savefig(MODELS + MODEL_NAME + '/'+ MODEL_NAME + '_ACCURACY.png')
#plt.savefig('./images/'+ MODEL_NAME + '_ACCURACY.png')
plt.show()
# Print out the score
#score = model_supclass_loaded.evaluate(val_pics , batch_size=1, verbose=1)
#print(score, model_supclass_loaded.metrics_names)

In [ ]:
model_history_df.describe()

In [ ]:
############################################################################################
# EVALUATION - LOSS
############################################################################################

plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history_df['loss'],label='Train loss')
plt.plot(model_history_df['val_loss'],label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
#plt.axis([0, 9, 0, 1000])
plt.legend()
#plt.savefig('./images/'+ MODEL_NAME + '_LOSS.png')
plt.show()

In [ ]:
eval_pics = tf.keras.preprocessing.image_dataset_from_directory(
    './data/food-101/images/',
    #'./data/iFood_2019/val_set/',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=1,
#    image_size=RESIZE_TO,
    shuffle=True,
    seed=SEED,
    validation_split=0.05,
    subset='validation',
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format='channels_last',
    verbose=True
)

In [ ]:
############################################################################################
# PREDICTION
############################################################################################



y_test_pred = np.array([],[])
y_test =  np.array([])

# batch=1
#for x, y in eval_pics:
#        y_test_pred = np.concatenate([y_test_pred, np.argmax(model_supclass_loaded.predict(x), axis = -1)])
#        y_test = np.concatenate([y_test, np.argmax(y.numpy(), axis=-1)])

# batches
with tf.device('/GPU:0'):
        for x, y in eval_pics:
                y_test_pred = np.concatenate([y_test_pred, np.argmax(model_supclass.predict(x, verbose=0), axis = -1)])
                y_test = np.concatenate([y_test, np.argmax(y.numpy(), axis=-1)])
                print(model_supclass_loaded.predict(x, verbose=0))

In [ ]:
#y_test_pred_df = pd.DataFrame(y_test_pred, columns=['label_predicted'])
#y_test_df = pd.DataFrame(y_test, columns=['label_true'])
#classes_df = pd.DataFrame(eval_pics.class_names, columns=['class_name'])

#y_test_df = y_test_df.apply(lambda x: classes_df['class_name'].iloc[x])
#y_test_pred_df = y_test_pred_df.apply(lambda x: classes_df['class_name'].iloc[x])

#y_test_df
print(y_test)
print(y_test_pred)

In [ ]:
############################################################################################
# PREDICTION - CONFMATRIX
############################################################################################

from sklearn.metrics import ConfusionMatrixDisplay
plt.rcParams['figure.figsize'] = (20, 20)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true= y_test, y_pred=y_test_pred)
#disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=np.unique(y_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.xticks(rotation=90)
#plt.savefig('./images/'+ MODEL_NAME + '_CONFMAT.png')
plt.show()






In [ ]:

eval_pics
#eval_pics.as_numpy_iterator().next()[0]
#eval_pics.take(1)

In [ ]:
#y_test_pred = np.concatenate([y_test_pred, np.argmax(model_supclass_loaded.predict(x, verbose=0), axis = -1)])
#y_test = np.concatenate([y_test, np.argmax(y.numpy(), axis=-1)])

In [ ]:
plt.figure(figsize=(15, 15))
for image, label in eval_pics.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title( classes_df['class_name'].iloc[np.argmax(labels[i].numpy(), axis=-1)]  )
        #plt.title( model_supclass_loaded.predict( tf.expand_dims(image, axis=0), verbose=0))  
        #plt.axis("off")
        

In [ ]:
#plt.imshow(images[0].numpy().astype("uint8"))
#plt.show()
